```{contents}
```
## Global State

In LangGraph, the **Global State** is the **central shared memory** that coordinates all computation across nodes, agents, tools, and control flow.
It is the **single source of truth** that drives execution, routing, decisions, persistence, and recovery.

---

### **1. Formal Definition**

> **Global State** is a **typed, persistent, versioned data structure** that is passed through every node of a LangGraph execution and incrementally updated as the graph runs.

Each node:

* **Reads** the current state
* **Computes** changes
* **Returns only the updates**
* LangGraph **merges** those updates into the global state

---

### **2. Why Global State Exists**

LLM workflows are not linear. They involve:

* Multiple agents
* Conditional routing
* Loops
* External tools
* Human intervention
* Long-running sessions

A centralized state provides:

| Requirement   | How Global State Helps              |
| ------------- | ----------------------------------- |
| Coordination  | All components see the same context |
| Memory        | Past actions influence future steps |
| Control flow  | Routing decisions depend on state   |
| Persistence   | Execution can be resumed            |
| Observability | Full execution trace                |

---

### **3. Structure of Global State**

The state is declared using a **Typed Schema**.

```python
from typing import TypedDict, List

class GraphState(TypedDict):
    messages: List[str]
    plan: str
    result: str
    step: int
    done: bool
```

This schema defines:

* What data exists
* How nodes can interact
* How updates are validated

---

### **4. State Lifecycle**

```
Initial Input
     ↓
Global State (created)
     ↓
Node Execution
     ↓
Partial Updates Returned
     ↓
Reducer Merges Updates
     ↓
New Global State
     ↓
Next Node
```

This repeats until the graph reaches `END`.

---

### **5. Partial State Updates**

Nodes **never overwrite the whole state**.

They return only what they modify.

```python
def planner(state: GraphState):
    return {"plan": "Search for recent papers"}
```

LangGraph merges:

```text
Old State + {"plan": "..."} → New Global State
```

---

### **6. State Reducers**

Reducers define **how updates are merged**.

```python
from langgraph.graph import add_messages

class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    step: int
```

Here:

* `messages` are **appended**
* `step` is overwritten

This enables fine-grained memory control.

---

### **7. Global State and Control Flow**

Routing decisions are pure functions of global state.

```python
def router(state: GraphState):
    if state["done"]:
        return END
    return "planner"
```

Thus, **state = program**.

---

### **8. Global State in Cyclic Graphs**

In loops, the state evolves over iterations:

```text
step=1 → step=2 → step=3 → done=True → exit
```

The loop continues until the global state satisfies the termination condition.

---

### **9. Persistence & Recovery**

LangGraph supports:

| Feature        | Description                  |
| -------------- | ---------------------------- |
| Checkpointing  | Save state at each step      |
| Thread ID      | Resume same execution        |
| Replay         | Reconstruct full history     |
| Crash Recovery | Restart from last checkpoint |

This makes Global State suitable for **production-grade workflows**.

---

### **10. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    count: int

def increment(state: State):
    return {"count": state["count"] + 1}

def stop_if_done(state: State):
    if state["count"] >= 3:
        return END
    return "increment"

builder = StateGraph(State)
builder.add_node("increment", increment)
builder.set_entry_point("increment")
builder.add_conditional_edges("increment", stop_if_done, {
    "increment": "increment",
    END: END
})

graph = builder.compile()
print(graph.invoke({"count": 0}))
```

---

### **11. Mental Model**

Global State behaves like:

> **Memory + Control Plane + Program Variables + Execution History**

All in one object.

Without global state, LangGraph would be a simple pipeline.
With it, LangGraph becomes a **full autonomous execution system**.

---

### **12. Common Design Patterns Using Global State**

| Pattern           | Usage                                 |
| ----------------- | ------------------------------------- |
| ReAct Loop        | Store thoughts, actions, observations |
| Planner–Executor  | Store plan and execution progress     |
| Multi-Agent       | Shared coordination memory            |
| Human-in-the-loop | Store approvals and feedback          |
| Self-Healing      | Store error history and fixes         |

### Demonstration

In [1]:
from typing import TypedDict, List
from typing_extensions import Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    messages: Annotated[List[str], add_messages]
    plan: str
    step: int
    done: bool


In [2]:
def planner(state: State):
    plan = "Research → Summarize → Deliver"
    return {
        "plan": plan,
        "messages": ["Plan created"],
        "step": 1
    }


In [3]:
def executor(state: State):
    msg = f"Executing step {state['step']} of plan"
    return {
        "messages": [msg],
        "step": state["step"] + 1
    }


In [5]:
from langgraph.graph import END

def router(state: State):
    if state["step"] > 3:
        return END
    return "executor"


In [6]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("planner", planner)
builder.add_node("executor", executor)

builder.set_entry_point("planner")
builder.add_edge("planner", "executor")

builder.add_conditional_edges("executor", router, {
    "executor": "executor",
    END: END
})

graph = builder.compile()


In [7]:
final_state = graph.invoke({
    "messages": [],
    "plan": "",
    "step": 0,
    "done": False
})

print(final_state)


{'messages': [HumanMessage(content='Plan created', additional_kwargs={}, response_metadata={}, id='227f33e4-150d-4a3f-be38-b1794d78c62d'), HumanMessage(content='Executing step 1 of plan', additional_kwargs={}, response_metadata={}, id='3c643561-e566-4416-9cb0-c57c27adda5b'), HumanMessage(content='Executing step 2 of plan', additional_kwargs={}, response_metadata={}, id='b992d5c4-8e3e-43e0-a5cd-c4195dac37e3'), HumanMessage(content='Executing step 3 of plan', additional_kwargs={}, response_metadata={}, id='c97e097a-16e9-4289-9fc7-9e44c4ccf4fa')], 'plan': 'Research → Summarize → Deliver', 'step': 4, 'done': False}
